In [32]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

#code found on stack exchange
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)
#

client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [33]:
tweetdf = dd.read_csv(urlpath='sample_mental_health_tweets.csv', sep='\t' )

countdf = dd.read_csv(urlpath='sample_mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 graph layer
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 graph layer


In [34]:

tweetdf['time'] = dd.to_datetime(tweetdf['time'])

countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

,timestamp,count,month,day,hour
0,2022-11-07 00:00:00+00:00,4088,11,7,0
1,2022-11-07 01:00:00+00:00,3974,11,7,1
2,2022-11-07 02:00:00+00:00,3839,11,7,2
3,2022-11-07 03:00:00+00:00,3631,11,7,3
4,2022-11-07 04:00:00+00:00,3249,11,7,4
...,...,...,...,...,...
115,2022-11-11 19:00:00+00:00,4897,11,11,19
116,2022-11-11 20:00:00+00:00,4884,11,11,20
117,2022-11-11 21:00:00+00:00,4591,11,11,21
118,2022-11-11 22:00:00+00:00,4293,11,11,22
